In [33]:
import pandas as pd
from datetime import datetime

In [39]:
wind_tunnel_data = "1_2_22/Motor testing 02_02_22 am.xlsx"
esc_data = "1_2_22/2022-02-01_17-33-14.csv"
export_name = "propulsion_3.xlsx"

In [40]:
wind_tunnel = pd.read_excel(wind_tunnel_data, engine='openpyxl')
wind_tunnel = wind_tunnel.iloc[2:]
wind_tunnel["Time"] = wind_tunnel["Seconds"].apply(lambda x: datetime.fromtimestamp(x))
wind_tunnel["Time"] = wind_tunnel["Time"].apply(lambda x: x.replace(year=2021, month=12 , day=18))
wind_tunnel["Time"] = pd.to_datetime(wind_tunnel["Time"]) 
wind_tunnel = wind_tunnel.set_index("Time")
wind_tunnel.columns = ['Date', 'Centre', 'Seconds',"Lift","Pitch","Drag","Side-Force","Dem Velocity","Meas Velocity","Temp"]
wind_tunnel=wind_tunnel.drop(columns=['Date', 'Centre', 'Seconds'])

ValueError: Length mismatch: Expected axis has 7 elements, new values have 10 elements

In [36]:
esc = pd.read_csv(esc_data)
esc["Time"] = esc["ms_today"].apply(lambda x: datetime.fromtimestamp(x/1000))
esc["Time"] = esc["Time"].apply(lambda x: x.replace(year=2021, month=12 , day=18))
esc["Time"] = pd.to_datetime(esc["Time"])
esc["rpm"] = esc["erpm"]/14
esc = esc.set_index("Time")
esc = esc.drop(["temp_mos_1","temp_mos_2","temp_mos_3","d_axis_current","q_axis_current","amp_hours_charged",
"watt_hours_charged","encoder_position","fault_code","vesc_id","d_axis_voltage","q_axis_voltage",
"amp_hours_charged_setup","watt_hours_charged_setup", "ms_today", "ms_today_setup", "temp_motor", "tachometer", "tachometer_abs", "battery_level",
"roll","pitch","yaw","accX","accY","accZ","gyroX","gyroY","gyroZ","gnss_posTime","gnss_lat","gnss_lon","gnss_alt","gnss_gVel","gnss_vVel","gnss_hAcc","gnss_vAcc"], axis=1)


In [37]:
esc_avg = esc.groupby(pd.Grouper(level='Time', freq='1s')).mean()
all_data = pd.concat([esc_avg, wind_tunnel],axis=1)
all_data= all_data.dropna(subset=["Lift", "Pitch","Drag","Side-Force","Dem Velocity","Meas Velocity","Temp"])
all_data=all_data.dropna(subset=esc.columns)
all_data.to_excel(export_name)

In [38]:
all_data

,input_voltage,temp_mos_max,current_motor,current_in,erpm,duty_cycle,amp_hours_used,watt_hours_used,amp_hours_setup,watt_hours_setup,...,num_vescs,ms_today_imu,rpm,Lift,Pitch,Drag,Side-Force,Dem Velocity,Meas Velocity,Temp
Time,,,,,,,,,,,,,,,,,,,,,
2021-12-18 17:33:14,23.176923,23.876923,0.026923,0.000000,1920.153846,0.023731,0.007500,0.173500,0.006923,0.160154,...,0.923077,-1.0,137.153846,-2.1787,-0.15847,0.096722,-0.040245,0,0,20.0879
2021-12-18 17:33:16,23.265517,23.741379,0.096897,0.000000,7.137931,0.000000,0.007500,0.173500,0.007500,0.173500,...,1.000000,-1.0,0.509852,-2.1872,-0.14031,0.14408,-0.018502,0,0,20.0872
2021-12-18 17:33:18,23.251852,23.755556,0.095926,0.000000,2.370370,0.000000,0.007500,0.173500,0.007500,0.173500,...,1.000000,-1.0,0.169312,-2.1954,-0.14399,0.14823,-0.029795,0,0,20.0805
2021-12-18 17:33:20,23.100000,23.872727,2.087576,0.200303,1004.545455,0.022788,0.007500,0.173600,0.007500,0.173573,...,1.000000,-1.0,71.753247,-2.1772,-1.0578,-1.8073,0.056232,0,0,20.1126
2021-12-18 17:33:22,22.985294,24.341176,9.858824,3.857353,27788.147059,0.390735,0.008906,0.207091,0.008876,0.206359,...,1.000000,-1.0,1984.867647,-2.3972,-3.376,-6.5514,-0.13024,0,0,20.0854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-18 18:22:35,23.552500,28.770000,29.525000,20.148250,45368.300000,0.682375,0.658512,15.440818,0.658367,15.437498,...,1.000000,-1.0,3240.592857,-2.4302,4.3901,8.8246,6.9568,0,3.46,20.0116
2021-12-18 18:22:37,23.696970,29.421212,19.126667,12.965152,39140.545455,0.556576,0.669494,15.699927,0.669385,15.697358,...,1.000000,-1.0,2795.753247,-1.4878,3.2169,6.7083,5.4884,0,1.61,20.0147
2021-12-18 18:22:39,24.181250,29.384375,0.851875,0.032188,3915.156250,0.042625,0.670600,15.725600,0.670600,15.725594,...,1.000000,-1.0,279.654018,-2.1331,-0.17626,0.28055,0.067768,0,0,20.0204
